In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split

In [35]:

EPOCHS = 20
IMG_WIDTH = 40
IMG_HEIGHT = 40
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


In [6]:
def process_image(img):
    #apply filters to improve image features
    #normalize the image
    img = img - np.mean(img)
    #sharpen the image
    avg_img = cv2.boxFilter(img, 3, (3,3))
    f = img - avg_img
    return f

In [7]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    labels = []
    images= []
    for root, _, filenames in os.walk(data_dir):
        for file in filenames:
            img = cv2.imread(os.path.join(root,file))
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            img = process_image(img)
            label = root.split('/').pop()
            images.append(img)
            labels.append(label)
    #print(labels[0])
    return (images, labels)
    raise NotImplementedError

In [8]:
def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(
        32, 3, activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    ))
    model.add(layers.Conv2D(
        32, 3, activation='relu', padding='valid'
    ))
   
    model.add(layers.MaxPooling2D(
        pool_size=(2,2), strides=(1,1), padding='valid', 
    ))
    
    model.add(layers.Dropout(0.25))
    
  
    
    model.add(layers.Conv2D(
        64, 3, activation='relu', padding='same'
    ))
    model.add(layers.Conv2D(
        64, 3, activation='relu', padding='same'
    ))
    
    model.add(layers.MaxPooling2D(
        pool_size=(2,2), strides=(1,1), padding='valid'
    ))
    model.add(layers.Dropout(0.25))
    model.add(
        layers.Flatten()
    )
    
    
    model.add(layers.Dense(
        units=500, activation='relu', use_bias=True, 
    ))
    model.add(layers.Dense(
        units=500, activation='relu', use_bias=True, 
    ))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(units=43, activation='softmax'))
    
    model.compile(optimizer='adam', 
                  loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy']
                 )
    return model
    raise NotImplementedError

In [63]:
def Alex_Net_model():
    """
    returns the compiled CNN model similar to Alexa Net
    """
    model = keras.Sequential()
    model.add(layers.Conv2D(
        8, 3, activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    ))
    model.add(layers.Conv2D(
        8, 3, activation='relu', padding='valid'
    )
    )
    model.add(layers.MaxPooling2D(
        pool_size=(2,2), strides=(1,1), padding='same'
    ))
    model.add(layers.Dropout(0.15))
    model.add(layers.Conv2D(
        32, 3, activation='relu', padding='valid'
    ))
    model.add(layers.Conv2D(
        32, 3, activation='relu', padding='valid'
    ))
    model.add(layers.MaxPooling2D(
        pool_size=(2,2), strides=(1,1), padding='same'
    ))
    model.add(layers.Conv2D(
        32, 3, activation='relu', padding='valid'
    ))
    model.add(layers.MaxPooling2D(
        pool_size=(2,2), strides=(1,1), padding='same'
    ))
    model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=235, activation='relu'
    ))
    model.add(layers.Dense(
        units=235, activation='relu'
    ))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(
        units=43, activation='softmax'
    ))
    model.compile(
        optimizer='adam',
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    return model
    

In [10]:
images, labels = load_data('/kaggle/input/traffic-data/gtsrb/')
print("Data Loaded successfully!")
# Split data into training and testing sets
labels = tf.keras.utils.to_categorical(labels)
x_train, x_test, y_train, y_test = train_test_split(
    np.array(images), np.array(labels), test_size=TEST_SIZE
)


Data Loaded successfully!


In [74]:
model = get_model()
# Fit model on training data
model.fit(x_train, y_train, epochs=EPOCHS)
# Evaluate neural network performance
print(model.evaluate(x_test,  y_test, verbose=2))

Epoch 1/20
500/500 [==============================] - 15s 29ms/step - loss: 7.6572 - accuracy: 0.0834
Epoch 2/20
500/500 [==============================] - 14s 29ms/step - loss: 0.9132 - accuracy: 0.7381
Epoch 3/20
500/500 [==============================] - 14s 29ms/step - loss: 0.1839 - accuracy: 0.9459
Epoch 4/20
500/500 [==============================] - 14s 29ms/step - loss: 0.1100 - accuracy: 0.9703
Epoch 5/20
500/500 [==============================] - 14s 29ms/step - loss: 0.0744 - accuracy: 0.9787
Epoch 6/20
500/500 [==============================] - 14s 29ms/step - loss: 0.0769 - accuracy: 0.9788
Epoch 7/20
500/500 [==============================] - 14s 29ms/step - loss: 0.0454 - accuracy: 0.9872
Epoch 8/20
500/500 [==============================] - 14s 29ms/step - loss: 0.0550 - accuracy: 0.9869
Epoch 9/20
500/500 [==============================] - 14s 29ms/step - loss: 0.0520 - accuracy: 0.9874
Epoch 10/20
500/500 [==============================] - 14s 29ms/step - loss: 0.087

In [75]:
model.summary()
model.save('Model1')

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_120 (Conv2D)          (None, 38, 38, 32)        896       
_________________________________________________________________
conv2d_121 (Conv2D)          (None, 36, 36, 32)        9248      
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 35, 35, 32)        0         
_________________________________________________________________
dropout_72 (Dropout)         (None, 35, 35, 32)        0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 35, 35, 64)        18496     
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 35, 35, 64)        36928     
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 34, 34, 64)      

In [64]:
alex_model = Alex_Net_model()
# Fit model on training data
alex_model.fit(x_train, y_train, epochs=EPOCHS)
# Evaluate neural network performance
print(alex_model.evaluate(x_test,  y_test, verbose=2))

Epoch 1/20
500/500 [==============================] - 6s 11ms/step - loss: 2.7744 - accuracy: 0.3911
Epoch 2/20
500/500 [==============================] - 5s 11ms/step - loss: 0.4698 - accuracy: 0.8698
Epoch 3/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2522 - accuracy: 0.9282
Epoch 4/20
500/500 [==============================] - 5s 11ms/step - loss: 0.1575 - accuracy: 0.9533
Epoch 5/20
500/500 [==============================] - 5s 11ms/step - loss: 0.1087 - accuracy: 0.9701
Epoch 6/20
500/500 [==============================] - 5s 11ms/step - loss: 0.0963 - accuracy: 0.9705
Epoch 7/20
500/500 [==============================] - 5s 11ms/step - loss: 0.0751 - accuracy: 0.9784
Epoch 8/20
500/500 [==============================] - 5s 11ms/step - loss: 0.0952 - accuracy: 0.9748
Epoch 9/20
500/500 [==============================] - 5s 10ms/step - loss: 0.0498 - accuracy: 0.9864
Epoch 10/20
500/500 [==============================] - 5s 11ms/step - loss: 0.0703 - accura

In [83]:
alex_model.summary()
alex_model.save('Alex_Model_2')


Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_115 (Conv2D)          (None, 38, 38, 8)         224       
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 36, 36, 8)         584       
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 36, 36, 8)         0         
_________________________________________________________________
dropout_69 (Dropout)         (None, 36, 36, 8)         0         
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 34, 34, 32)        2336      
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 32, 32, 32)      

In [82]:
!rm -r ./Alex_Model_2